# Notebook Summary 


### Quickstart

  1. Import etiq library - for install please check our docs (https://docs.etiq.ai/) 

  2. Login to the dashboard - this way you can send the results to your dashboard instance (Etiq AWS instance if you use the SaaS version). To deploy on your own cloud instance, get in touch (info@etiq.ai)

  3. Create or open a project 
  
  
  
### Snapshot 1, already-trained model 


  7. Load Adult dataset
  
  8. Train a model 
  
  9. Load your config file and create your snapshot using the test dataset only 
  
  10. Scan for bias rca issues 
  
 
  

## What are Bias RCA scans?

Some scans are simple tests that show whether a metric is above or below certain thresholds set by the user. Other scans are more complex and look at whether for a part of the sample dataset a metric is below or above the threshold. This will help you discover segments of customers or groups of records for which the model has a lower than expected accuracy or groups for which bias thresholds are not met.

Imagine if your bias metrics tests have picked up issues, this test finds out exactly which segment has the issue, which should help you fix it sooner. If only a part of the data drifted or only a segment is underperforming, your overall tests might not pick up on it, but this test would. While you can pre-set segments you are interested in, if you just run the scan as is, it will discover problematic segments on its own.


The RCA bias scans so far provide the following metrics out-of-the-box:

1. Equal Opportunity: measures the difference in true positive rate between a privileged demographic group and an unprivileged demographic group.

2. Demographic Parity: measures the difference between number of positive labels out of total from a privileged demographic group vs. a unprivileged demographic group)

3. Equal Odds TNR: measures the difference between true negative rate - privileged vs. unprivileged. The full measure in the literature looks for an optimal point where the difference in true positive rate between demographic groups as well as the difference in true negative rate between demographic groups are both minimized.

4. Individual Fairness: measures whether individuals with similar features observe the same model responses

This test pipeline is experimental. 

# SET-UP

In [1]:
import etiq


Thanks for trying out the ETIQ.ai toolkit!

Visit our getting started documentation at https://docs.etiq.ai/

Visit our Slack channel at https://etiqcore.slack.com/ for support or feedback.



In [2]:
from etiq import login as etiq_login
etiq_login("https://dashboard.etiq.ai/", <token>)


SyntaxError: invalid syntax (3951993118.py, line 2)

In [ ]:
# Can get/create a single named project
project = etiq.projects.open(name="Bias RCA Scans")

# SNAPSHOT 1: xgboost, pre-configured model


To illustrate some of the library's features, we build a model that predicts whether an applicant makes over or under 50K using the Adult dataset from https://archive.ics.uci.edu/ml/datasets/adult.


First, we'll be encoding the categorical features found in this dataset.

Second, we'll log the dataset to Etiq.

In this case we encode prior to splitting into test/train/validate because we know in advance the categories people fall into for this dataset. This means that in production we won't run into new categories that will fall into a bucket not included in this dataset, This allows us to encode prior to splitting into train/test/validation.

However if this is not the case for your use case, you should NOT encode prior to splitting your sample, as this might lead to LEAKAGE. Encoding categorical values itself is problematic as it assigns a numerical ranking to categorical variables. For best practice encoding use one hot encoding. This is for example purposes

Once model is built, we will:
 - log the relevant config to etiq 
 - log the model together with the hold-out sample to etiq
 - run the accuracy rca scan



## Model Build 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Loading a dataset. We're using the adult dataset
data = etiq.utils.load_sample("adultdata")
data.head()



In [ ]:
# use a LabelEncoder to transform categorical variables
cont_vars = ['age', 'educational-num', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_vars = list(set(data.columns.values) - set(cont_vars))

label_encoders = {}
data_encoded = pd.DataFrame()
for i in cat_vars:
    label = LabelEncoder()
    data_encoded[i] = label.fit_transform(data[i])
    label_encoders[i] = label

data_encoded.set_index(data.index, inplace=True)
data_encoded = pd.concat([data.loc[:, cont_vars], data_encoded], axis=1).copy()



In [ ]:
# prepare the training/testing/validation datasets

# separate into train/validate/test dataset of sizes 80%/10%/10% as percetages of the initial data
data_remaining, test = train_test_split(data_encoded, test_size=0.1)
train, valid = train_test_split(data_remaining, test_size=0.1112)

# because we don't want to train on protected attributes or labels to be predicted, 
# let's remove these columns from the training dataset
protected_train = train['gender'].copy() # gender is a protected attribute
y_train = train['income'].copy() # labels we're going to train the model to predict
x_train = train.drop(columns=['gender','income'])
protected_valid = valid['gender'].copy() 
y_valid = valid['income'].copy() 
x_valid = valid.drop(columns=['gender','income'])
protected_test = test['gender'].copy() 
y_test = test['income'].copy()
x_test = test.drop(columns=['gender','income'])

In [ ]:
# train a XGBoost model to predict 'income'

standard_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=4)    
model_fit = standard_model.fit(x_train, y_train)

In [ ]:
y_train_pred = standard_model.predict(x_train)
y_valid_pred = standard_model.predict(x_valid)
print('Model accuracy on the training dataset :', 
      round(100 * accuracy_score(y_train, y_train_pred),2),'%') # round the score to 2 digits  

print('Model accuracy on the validation dataset :', 
      round(100 * accuracy_score(y_valid, y_valid_pred),2),'%')

## Add a Custom Metric

In [ ]:
@etiq.metrics.bias_metric
@etiq.custom_metric
@etiq.actual_values("actual")
@etiq.prediction_values("predictions")
@etiq.positive_outcome("positive_outcome_label")
@etiq.negative_outcome("negative_outcome_label")

def treatment_equality(predictions, actual, positive_outcome_label, negative_outcome_label):
    false_neg = sum((predictions != actual) & (predictions == negative_outcome_label))
    false_pos = sum((predictions != actual) & (predictions == positive_outcome_label))
    
    return false_pos/false_neg - false_pos/false_neg

## Log config, dataset and model to Etiq

For already trained models make sure you only you use a sample you held out. 

As you don't want any retraining of the model to occur, set your train_valid_test split to [0.0, 1.0, 0.0]. 

In this instance we have used a low volume for the minimum segment size to illustrate the issues, but you should probably set it higher, depending on your use case and sample size. Also we have set the threshold for bias low so as to make sure it finds at least one issue.

In [ ]:
from etiq import Model

with etiq.etiq_config("./config_bias_rca.json"):
    #log your dataset (the sample you held-out!!!)
    dataset = etiq.BiasDatasetBuilder.dataset(test)
    
    #Log your already trained model

    model = Model(model_fitted=model_fit)
    
    # Create the snapshot
    snapshot = project.snapshots.create(name="Snapshot 1", 
                                        dataset=dataset,
                                        model=model, 
                                        bias_params=etiq.biasparams.BiasParams(protected='gender', 
                                                                               privileged=1, 
                                                                               unprivileged=0, 
                                                                               positive_outcome_label=1, 
                                                                               negative_outcome_label=0)
                                       )
    print("Running Bias Scan... \n")
    (segments_bias, issues_bias, issue_summary_bias) = snapshot.scan_bias_metrics()
    print("\n Running RCA Bias Scan... \n")
    (segments_bias_rca, issues_bias_rca, issue_summary_bias_rca)  = snapshot.scan_bias_metrics_rca()

In [ ]:
issue_summary_bias

In [ ]:
issue_summary_bias_rca

In [ ]:
issues_bias_rca

In [ ]:
segments_bias_rca